# Importações


In [2]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import plotly.express as px
from fuzzywuzzy import process

# Conexão com o banco de dados


In [3]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

# Leitura do banco de dados


In [4]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

# Query para o banco de dados


In [5]:
# cSpell: words automacao, ocorrencia
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> tuple:
        """
        Realiza a leitura dos dados do banco de dados.
        Retorna na ordem: df_occ, df_info, df_cadastro
        """

        # Dia de hoje
        now = pd.to_datetime("today")

        # Encontrando primeiro dia do mês atual
        first_day = now.replace(day=1)

        # Mantendo apenas a data
        first_day = first_day.strftime("%Y-%m-%d")

        # Query para leitura dos dados de ocorrência
        query_occ = self.db_read.create_automacao_query(
            table="maquina_ocorrencia",
            where=f"data_registro >= '{first_day}'",
        )

        query_info = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha,"
            " (SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica,"
            " t1.status,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " t1.data_registro,"
            " t1.hora_registro"
            " FROM "
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro >= '{first_day}'"
            " ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
        )

        query_production = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro desc) as linha,"
            " t1.turno,"
            " MAX(t1.contagem_total_ciclos) total_ciclos,"
            " MAX(t1.contagem_total_produzido) total_produzido,"
            " t1.data_registro"
            " FROM"
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro >= '{first_day}'"
            " GROUP BY t1.maquina_id, t1.data_registro, t1.turno"
            " ORDER BY data_registro DESC, maquina_id, turno"
        )

        print("========== Baixando dados do DB ==========")

        # Leitura dos dados
        df_occ = self.db_read.get_automacao_data(query_occ)
        df_info = self.db_read.get_automacao_data(query_info)
        df_info_production = self.db_read.get_automacao_data(query_production)

        # Verificando se os dados foram lidos corretamente
        if df_occ.empty or df_info.empty or df_info_production.empty:
            print("====== Erro na leitura dos dados ======")
            return None, None, None

        print("Ok...")

        return df_occ, df_info, df_info_production


get_data = GetData()
df_occ, df_info, df_info_production = get_data.get_data()

========== Baixando dados do DB ==========


Ok...


## Testes de saída do banco de dados


In [6]:
df_occ

,recno,maquina_id,motivo_id,problema,solucao,data_registro,hora_registro,usuario_id
0,2638,TMF006,12,,,2024-01-02,01:16:52,000453
1,2639,TMF013,08,,,2024-01-02,01:17:04,000453
2,2640,TMF010,08,,,2024-01-02,01:17:20,000453
3,2641,TMF010,12,,,2024-01-02,01:18:45,000453
4,2642,TMF006,12,,,2024-01-03,08:14:58,000807
...,...,...,...,...,...,...,...,...
669,3307,TMF012,11,Setup para bolinha 300g,,2024-01-29,19:29:05,000939
670,3308,TMF007,12,,,2024-01-30,04:28:04,000453
671,3309,TMF010,12,,,2024-01-30,04:28:23,000453
672,3310,TMF008,03,,,2024-01-30,04:28:48,000453


In [7]:
df_info

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF001,9,1,true,MAT,10.0,10.0,2024-01-30,08:01:30.960000
1,TMF003,8,1,true,MAT,10.0,10.0,2024-01-30,08:01:29.956666
2,TMF009,7,1,true,MAT,8.0,6.0,2024-01-30,08:01:28.956666
3,TMF004,6,1,false,MAT,0.0,0.0,2024-01-30,08:01:27.956666
4,TMF014,5,1,true,MAT,8.0,8.0,2024-01-30,08:01:26.953333
...,...,...,...,...,...,...,...,...,...
282958,TMF014,5,1,false,VES,0.0,0.0,2024-01-01,00:00:18.253333
282959,TMF011,4,1,false,VES,0.0,0.0,2024-01-01,00:00:17.253333
282960,TMF015,3,1,false,VES,0.0,0.0,2024-01-01,00:00:16.253333
282961,TMF002,2,1,false,VES,0.0,0.0,2024-01-01,00:00:15.253333


In [8]:
df_info_production

,maquina_id,linha,turno,total_ciclos,total_produzido,data_registro
0,TMF001,9,MAT,10.0,10.0,2024-01-30
1,TMF001,9,NOT,1918.0,1862.0,2024-01-30
2,TMF002,2,MAT,8.0,8.0,2024-01-30
3,TMF002,2,NOT,8462.0,8420.0,2024-01-30
4,TMF003,8,MAT,10.0,10.0,2024-01-30
...,...,...,...,...,...,...
1216,TMF014,5,NOT,0.0,0.0,2024-01-01
1217,TMF014,5,VES,0.0,0.0,2024-01-01
1218,TMF015,3,MAT,0.0,0.0,2024-01-01
1219,TMF015,3,NOT,0.0,0.0,2024-01-01


# Limpeza de dados e análise exploratória


In [9]:
# service/clean_data.py


# cSpell: disable=invalid-name
class CleanData:
    def maq_info(self, info: pd.DataFrame) -> pd.DataFrame:
        """
        Processa as informações de uma máquina e retorna um DataFrame com os dados ajustados.

        Args:
            info (pd.DataFrame): DataFrame contendo as informações da máquina.

        Returns:
            pd.DataFrame: DataFrame com os dados ajustados da máquina.
        """

        # Ordenar dataframe
        df_info = info.sort_values(by=["maquina_id", "data_registro", "hora_registro", "turno"])

        # Criar coluna com data e hora unidos
        df_info["data_hora_registro"] = (
            df_info["data_registro"].astype(str)
            + " "
            + df_info["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Ajustar primeira entrada se for VES
        mask = (df_info["turno"] == "VES") & (
            df_info["maquina_id"] != df_info["maquina_id"].shift()
        )
        df_info["turno"] = np.where(mask, "NOT", df_info["turno"])

        # Ajustar data_hora para pd.datetime
        df_info["data_hora_registro"] = pd.to_datetime(df_info["data_hora_registro"])

        # Ajustar horário se turno for VES - ajusta para dia anterior e horário 23:59:59
        mask = (
            (df_info["turno"] == "VES")
            & (df_info["data_hora_registro"] != df_info["data_hora_registro"].shift())
            & (df_info["data_hora_registro"].dt.time > time(0, 0, 0))
            & (df_info["data_hora_registro"].dt.time < time(0, 5, 0))
        )
        df_info["data_hora_registro"] = np.where(
            mask,
            (df_info["data_hora_registro"] - pd.Timedelta(days=1)).dt.normalize()
            + pd.Timedelta(hours=23, minutes=59, seconds=59),
            df_info["data_hora_registro"],
        )

        # Criar nova coluna status_change para identificar mudança de status
        df_info["status_change"] = df_info["status"].ne(df_info["status"].shift())

        # Criar coluna para identificar a mudança de máquina
        df_info["maquina_change"] = df_info["maquina_id"].ne(df_info["maquina_id"].shift())

        # Criar coluna para identificar a mudança de turno
        df_info["turno_change"] = df_info["turno"].ne(df_info["turno"].shift())

        # Atualizar coluna change para incluir mudança de turno
        df_info["change"] = (
            df_info["status_change"] | df_info["maquina_change"] | df_info["turno_change"]
        )

        # Agrupar por maquina e identificar data e hora da última mudança de status
        df_info["change_time"] = (
            df_info.groupby("maquina_id")["data_hora_registro"].shift(0).where(df_info["change"])
        )

        # Feito para agrupar por maquina_id e turno e manter o ultimo registro de cada grupo
        df_info = (
            df_info.groupby(["maquina_id", "change_time"])
            .agg(
                status=("status", "first"),
                turno=("turno", "first"),
                linha=("linha", "first"),
                fabrica=("fabrica", "first"),
                data_hora_registro=("data_hora_registro", "first"),
                contagem_total_ciclos=("contagem_total_ciclos", "last"),
                contagem_total_produzido=(
                    "contagem_total_produzido",
                    "last",
                ),
                change=("change", "first"),
                maquina_change=("maquina_change", "first"),
            )
            .reset_index()
        )

        # Criar nova coluna com a data_hora_final do status
        df_info["data_hora_final"] = (
            df_info.groupby("maquina_id")["data_hora_registro"]
            .shift(-1)
            .where(~df_info["maquina_change"])
        )

        # Atualizar coluna data_hora_final onde maquina_change é True
        mask = df_info["maquina_change"]
        df_info["data_hora_final"] = np.where(
            mask, df_info["change_time"].shift(-1), df_info["data_hora_final"]
        )

        # Remover colunas desnecessárias
        df_info.drop(
            columns=[
                "maquina_change",
                "change",
                "change_time",
            ],
            inplace=True,
        )

        # Remover linhas onde data_hora_final é nulo
        df_info.dropna(subset=["data_hora_final"], inplace=True)

        # Cria nova coluna tempo_registro_min para calcular o tempo de registro em minutos
        df_info["tempo_registro_min"] = (
            pd.to_datetime(df_info["data_hora_final"])
            - pd.to_datetime(df_info["data_hora_registro"])
        ).dt.total_seconds() / 60

        # Arredondar tempo_registro_min e converter para inteiro
        df_info["tempo_registro_min"] = df_info["tempo_registro_min"].round(0).astype(int)

        # Ajustar tipos
        df_info = df_info.astype(
            {
                "maquina_id": "category",
                "status": "category",
                "turno": "category",
                "linha": "category",
                "fabrica": "category",
                "tempo_registro_min": int,
                "contagem_total_ciclos": int,
                "contagem_total_produzido": int,
            }
        )

        # Ajustar nomenclatura dos status
        df_info["status"] = np.where(
            (df_info["status"] == "true") & (df_info["tempo_registro_min"] < 10),
            "in_test",
            df_info["status"],
        )
        df_info["status"] = np.where(df_info["status"] == "true", "rodando", df_info["status"])
        df_info["status"] = np.where(df_info["status"] == "false", "parada", df_info["status"])

        # Ajustar tipo do status
        df_info["status"] = df_info["status"].astype("category")

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def get_adjusted_stops_data(self, info: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de paradas ajustados de acordo com as regras definidas.

        Args:
            info (pd.DataFrame): O dataframe contendo os dados de paradas.

        Returns:
            pd.DataFrame: O dataframe com os dados de paradas ajustados.
        """
        # Certificar que data_hora_registro e data_hora_final são do tipo datetime
        info["data_hora_registro"] = pd.to_datetime(info["data_hora_registro"])
        info["data_hora_final"] = pd.to_datetime(info["data_hora_final"])

        # Ordenar por maquina_id e data_hora_registro
        df_info = info.sort_values(by=["maquina_id", "data_hora_registro"])

        # Criar coluna auxiliar para identificar a maquina rodando
        df_info["rodando"] = np.where(df_info["status"] == "rodando", 1, 0)

        # Unir grupos de paradas, levando em conta mudança de maquina e turno
        df_info["group"] = (
            (df_info["rodando"] != df_info["rodando"].shift())
            | (df_info["maquina_id"] != df_info["maquina_id"].shift())
            | (df_info["turno"] != df_info["turno"].shift())
            | (
                df_info["data_hora_registro"].dt.date
                != df_info["data_hora_registro"].shift().dt.date
            )
        ).cumsum()

        # Agrerar por grupo
        df_info = (
            df_info.groupby(["group"])
            .agg(
                maquina_id=("maquina_id", "first"),
                status=("status", "first"),
                turno=("turno", "first"),
                linha=("linha", "first"),
                fabrica=("fabrica", "first"),
                data_hora_registro=("data_hora_registro", "first"),
                data_hora_final=("data_hora_final", "last"),
                tempo_registro_min=("tempo_registro_min", "sum"),
                contagem_total_ciclos=("contagem_total_ciclos", "last"),
                contagem_total_produzido=("contagem_total_produzido", "last"),
            )
            .reset_index(drop=True)
        )

        # Alterar in_test para parada
        df_info["status"] = np.where(df_info["status"] == "in_test", "parada", df_info["status"])

        # Substituir valores nulos por np.nan
        df_info.fillna(value=np.nan, inplace=True)

        return df_info

    def dayofweek_adjust(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Incluir colunas para identificar sábados, domingos e feriados.

        Args:
            df (pd.DataFrame): DataFrame com os dados de paradas.

        Returns:
            pd.DataFrame: DataFrame com as colunas adicionadas.
        """

        # Garantir que data_hora_registro é do tipo datetime
        df["data_hora_registro"] = pd.to_datetime(df["data_hora_registro"])

        # Identificar sábados
        df["sabado"] = np.where(df["data_hora_registro"].dt.dayofweek == 5, 1, 0)

        # Identificar domingos
        df["domingo"] = np.where(df["data_hora_registro"].dt.dayofweek == 6, 1, 0)

        # Ler arquivo com os feriados
        holidays = pd.read_csv("../assets/feriados.csv")

        # Converter para datetime
        holidays["feriados"] = pd.to_datetime(holidays["feriados"])

        # Identificar feriados
        df["feriado"] = (
            df["data_hora_registro"]
            .dt.date.isin(pd.to_datetime(holidays["feriados"]).dt.date)
            .astype(int)
        )

        return df

    def get_maq_info_cleaned(self, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de paradas ajustados de acordo com as regras definidas.

        Args:
            df_info (pd.DataFrame): O dataframe contendo os dados de paradas.

        Returns:
            pd.DataFrame: O dataframe com os dados de paradas ajustados.
        """

        # Ajustar dados de maquina_info
        df_info = self.maq_info(df_info)

        # Ajustar dados de paradas
        df_info = self.get_adjusted_stops_data(df_info)

        # Incluir colunas para identificar sábados, domingos e feriados
        df_info = self.dayofweek_adjust(df_info)

        return df_info

    def get_maq_occ_cleaned(self, df_occ: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de ocorrências ajustados de acordo com as regras definidas.

        Args:
            df_occ (pd.DataFrame): O dataframe contendo os dados de ocorrências.

        Returns:
            pd.DataFrame: O dataframe com os dados de ocorrências ajustados.
        """

        # Motivos de Parada
        motivos = {
            1: "Ajustes",
            2: "Troca de Bobina",
            3: "Refeição",
            4: "Reunião",
            5: "Café e Ginástica Laboral",
            6: "Limpeza",
            7: "Manutenção Elétrica",
            8: "Manutenção Mecânica",
            9: "Material em Falta",
            10: "Setup de Sabor",
            11: "Setup de Tamanho",
            12: "Parada Programada",
            13: "Intervenção de Qualidade",
            14: "Linha Cheia",
            15: "Treinamento",
            16: "Limpeza Industrial",
            17: "Troca de Filme",
        }

        # Ajustar coluna motivo_id para int
        df_occ = df_occ.astype({"motivo_id": int})

        # Unir data_registro e hora_registro
        df_occ["data_hora_registro"] = (
            df_occ["data_registro"].astype(str)
            + " "
            + df_occ["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Ajustar data_hora_registro para datetime
        df_occ["data_hora_registro"] = pd.to_datetime(df_occ["data_hora_registro"])

        # Criar coluna com motivo_nome com base no dicionário motivos
        df_occ["motivo_nome"] = df_occ["motivo_id"].map(motivos)

        # Ajustar "problema" e "solucao" se a string estiver vazia
        df_occ["problema"] = np.where(df_occ["problema"] == "", np.nan, df_occ["problema"])
        df_occ["solucao"] = np.where(df_occ["solucao"] == "", np.nan, df_occ["solucao"])

        # Copiar motivo_nome para problema caso problema seja nulo e motivo_id não seja 1,7,8,9,14
        df_occ["problema"] = np.where(
            (df_occ["problema"].isnull())
            & (
                ~df_occ["motivo_id"].isin(
                    [
                        1,
                        7,
                        8,
                        9,
                        14,
                    ]
                )
            ),
            df_occ["motivo_nome"],
            df_occ["problema"],
        )

        # Ajustar ordem das colunas e seus tipos
        df_occ = df_occ.reindex(
            columns=[
                "maquina_id",
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro",
                "usuario_id",
            ]
        )
        df_occ = df_occ.astype(
            {
                "maquina_id": "category",
                "motivo_id": int,
                "motivo_nome": "category",
                "problema": str,
                "solucao": "category",
                "data_hora_registro": "datetime64[ns]",
                "usuario_id": "category",
            }
        )

        return df_occ

    def get_maq_production_cleaned(self, df_production: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de produção ajustados de acordo com as regras definidas.

        Args:
            df_production (pd.DataFrame): O dataframe contendo os dados de produção.

        Returns:
            pd.DataFrame: O dataframe com os dados de produção ajustados.
        """

        # Incluir coluna turno_number para ordenar os turnos
        df_production["turno_number"] = df_production["turno"].map({"MAT": 2, "VES": 3, "NOT": 1})

        # Ordenar por maquina_id, data_registro e turno_number
        df_production.sort_values(by=["maquina_id", "data_registro", "turno_number"], inplace=True)

        # Remover coluna turno_number
        df_production.drop(columns=["turno_number"], inplace=True)

        # Ajustar tipos
        df_production = df_production.astype(
            {
                "maquina_id": "category",
                "linha": "category",
                "turno": "category",
                "total_ciclos": int,
                "total_produzido": int,
                "data_registro": "datetime64[ns]",
            }
        )

        # Ajustar o index
        df_production.reset_index(drop=True, inplace=True)

        return df_production


clean_data = CleanData()

In [10]:
df_info_clean = clean_data.maq_info(df_info.copy())
df_info_clean

,maquina_id,status,turno,linha,fabrica,data_hora_registro,contagem_total_ciclos,contagem_total_produzido,data_hora_final,tempo_registro_min
0,TMF001,parada,NOT,8,1,2024-01-01 00:00:21,0,0,2024-01-01 08:02:22,482
1,TMF001,parada,MAT,8,1,2024-01-01 08:02:22,0,0,2024-01-01 16:02:24,480
2,TMF001,parada,VES,8,1,2024-01-01 16:02:24,0,0,2024-01-02 00:02:25,480
3,TMF001,parada,NOT,8,1,2024-01-02 00:02:25,0,0,2024-01-02 08:02:27,480
4,TMF001,parada,MAT,8,1,2024-01-02 08:02:27,0,0,2024-01-02 16:02:28,480
...,...,...,...,...,...,...,...,...,...,...
11336,TMF015,rodando,VES,3,1,2024-01-29 23:31:23,5010,5002,2024-01-29 23:59:23,28
11337,TMF015,parada,VES,3,1,2024-01-29 23:59:23,5360,5352,2024-01-30 00:01:23,2
11338,TMF015,rodando,NOT,3,1,2024-01-30 00:01:23,8,8,2024-01-30 04:01:24,240
11339,TMF015,parada,NOT,3,1,2024-01-30 04:01:24,5310,5310,2024-01-30 05:03:24,62


In [11]:
df_info_stops = clean_data.get_adjusted_stops_data(df_info_clean.copy())
df_info_stops

,maquina_id,status,turno,linha,fabrica,data_hora_registro,data_hora_final,tempo_registro_min,contagem_total_ciclos,contagem_total_produzido
0,TMF001,parada,NOT,8,1,2024-01-01 00:00:21,2024-01-01 08:02:22,482,0,0
1,TMF001,parada,MAT,8,1,2024-01-01 08:02:22,2024-01-01 16:02:24,480,0,0
2,TMF001,parada,VES,8,1,2024-01-01 16:02:24,2024-01-02 00:02:25,480,0,0
3,TMF001,parada,NOT,8,1,2024-01-02 00:02:25,2024-01-02 08:02:27,480,0,0
4,TMF001,parada,MAT,8,1,2024-01-02 08:02:27,2024-01-02 16:02:28,480,0,0
...,...,...,...,...,...,...,...,...,...,...
7915,TMF015,rodando,VES,3,1,2024-01-29 23:31:23,2024-01-29 23:59:23,28,5010,5002
7916,TMF015,parada,VES,3,1,2024-01-29 23:59:23,2024-01-30 00:01:23,2,5360,5352
7917,TMF015,rodando,NOT,3,1,2024-01-30 00:01:23,2024-01-30 04:01:24,240,8,8
7918,TMF015,parada,NOT,3,1,2024-01-30 04:01:24,2024-01-30 05:03:24,62,5310,5310


In [12]:
df_info_cleaned = clean_data.get_maq_info_cleaned(df_info.copy())
df_info_cleaned

,maquina_id,status,turno,linha,fabrica,data_hora_registro,data_hora_final,tempo_registro_min,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,TMF001,parada,NOT,8,1,2024-01-01 00:00:21,2024-01-01 08:02:22,482,0,0,0,0,1
1,TMF001,parada,MAT,8,1,2024-01-01 08:02:22,2024-01-01 16:02:24,480,0,0,0,0,1
2,TMF001,parada,VES,8,1,2024-01-01 16:02:24,2024-01-02 00:02:25,480,0,0,0,0,1
3,TMF001,parada,NOT,8,1,2024-01-02 00:02:25,2024-01-02 08:02:27,480,0,0,0,0,0
4,TMF001,parada,MAT,8,1,2024-01-02 08:02:27,2024-01-02 16:02:28,480,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,TMF015,rodando,VES,3,1,2024-01-29 23:31:23,2024-01-29 23:59:23,28,5010,5002,0,0,0
7916,TMF015,parada,VES,3,1,2024-01-29 23:59:23,2024-01-30 00:01:23,2,5360,5352,0,0,0
7917,TMF015,rodando,NOT,3,1,2024-01-30 00:01:23,2024-01-30 04:01:24,240,8,8,0,0,0
7918,TMF015,parada,NOT,3,1,2024-01-30 04:01:24,2024-01-30 05:03:24,62,5310,5310,0,0,0


In [13]:
df_maq_occ_cleaned = clean_data.get_maq_occ_cleaned(df_occ.copy())
df_maq_occ_cleaned

,maquina_id,motivo_id,motivo_nome,problema,solucao,data_hora_registro,usuario_id
0,TMF006,12,Parada Programada,Parada Programada,NaN,2024-01-02 01:16:52,000453
1,TMF013,8,Manutenção Mecânica,nan,NaN,2024-01-02 01:17:04,000453
2,TMF010,8,Manutenção Mecânica,nan,NaN,2024-01-02 01:17:20,000453
3,TMF010,12,Parada Programada,Parada Programada,NaN,2024-01-02 01:18:45,000453
4,TMF006,12,Parada Programada,Parada Programada,NaN,2024-01-03 08:14:58,000807
...,...,...,...,...,...,...,...
669,TMF012,11,Setup de Tamanho,Setup para bolinha 300g,NaN,2024-01-29 19:29:05,000939
670,TMF007,12,Parada Programada,Parada Programada,NaN,2024-01-30 04:28:04,000453
671,TMF010,12,Parada Programada,Parada Programada,NaN,2024-01-30 04:28:23,000453
672,TMF008,3,Refeição,Refeição,NaN,2024-01-30 04:28:48,000453


In [14]:
df_maq_info_production_cleaned = clean_data.get_maq_production_cleaned(df_info_production.copy())
df_maq_info_production_cleaned

,maquina_id,linha,turno,total_ciclos,total_produzido,data_registro
0,TMF001,8,NOT,0,0,2024-01-01
1,TMF001,8,MAT,0,0,2024-01-01
2,TMF001,8,VES,0,0,2024-01-01
3,TMF001,8,NOT,0,0,2024-01-02
4,TMF001,8,MAT,0,0,2024-01-02
...,...,...,...,...,...,...
1216,TMF015,3,NOT,52,52,2024-01-29
1217,TMF015,3,MAT,3082,3082,2024-01-29
1218,TMF015,3,VES,5360,5352,2024-01-29
1219,TMF015,3,NOT,9240,9240,2024-01-30


# Unir info e ocorrências


In [19]:
# service/join_data.py
# cSpell: disable=invalid-name


class JoinData:
    """
    Essa classe é responsável por juntar os dados de ocorrências, paradas e produção.
    """

    def join_info_occ(self, df_occ: pd.DataFrame, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Junta os dados de ocorrências e paradas.

        Args:
            df_occ (pd.DataFrame): DataFrame com os dados de ocorrências.
            df_info (pd.DataFrame): DataFrame com os dados de paradas.

        Returns:
            pd.DataFrame: DataFrame com os dados de ocorrências e paradas juntos.
        """

        # Garantir que as culunas com datas sejam datetime
        df_occ["data_hora_registro"] = pd.to_datetime(df_occ["data_hora_registro"])
        df_info["data_hora_registro"] = pd.to_datetime(df_info["data_hora_registro"])
        df_info["data_hora_final"] = pd.to_datetime(df_info["data_hora_final"])

        # Juntar os dados de ocorrências e paradas
        def merge_rows(row):
            """
            Função para juntar os dados de ocorrências e paradas.
            """
            mask = (
                (df_occ["maquina_id"] == row["maquina_id"])
                & (df_occ["data_hora_registro"] >= row["data_hora_registro"])
                & (df_occ["data_hora_registro"] <= row["data_hora_final"])
            )  # mask para identificar as paradas que ocorreram durante a ocorrência

            # Criar dataframe com as paradas que ocorreram durante a ocorrência
            if df_occ[mask].shape[0] > 0:
                return pd.Series(
                    [
                        df_occ[mask]["motivo_id"].values[0],
                        df_occ[mask]["motivo_nome"].values[0],
                        df_occ[mask]["problema"].values[0],
                        df_occ[mask]["solucao"].values[0],
                        df_occ[mask]["data_hora_registro"].values[0],
                        df_occ[mask]["usuario_id"].values[0],
                    ],
                )
            else:
                return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])

        # Aplicar a função merge_rows
        df_info[
            [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro_occ",
                "usuario_id_occ",
            ]
        ] = df_info.apply(merge_rows, axis=1)

        # Reordenar colunas
        df_info = df_info.reindex(
            columns=[
                "maquina_id",
                "linha",
                "fabrica",
                "turno",
                "data_hora_registro",
                "tempo_registro_min",
                "data_hora_final",
                "status",
                "motivo_id",
                "data_hora_registro_occ",
                "motivo_nome",
                "problema",
                "solucao",
                "usuario_id_occ",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "sabado",
                "domingo",
                "feriado",
            ]
        )

        # Ajustar problema e solução caso seja "nan"
        df_info["problema"] = np.where(df_info["problema"] == "nan", np.nan, df_info["problema"])
        df_info["solucao"] = np.where(df_info["solucao"] == "nan", np.nan, df_info["solucao"])

        # Remove a linha com tempo_registro_min negativo ou 0
        df_info = df_info[df_info["tempo_registro_min"] > 0]

        def move_columns(df: pd.DataFrame) -> pd.DataFrame:
            # Adiciona uma verificação para garantir que 'data_hora_registro_occ', 'data_hora_registro' e 'data_hora_final' não sejam NaN
            mask = (
                df[["data_hora_registro_occ", "data_hora_registro", "data_hora_final"]]
                .notna()
                .all(axis=1)
            )

            df["data_hora_registro_occ"] = df["data_hora_registro_occ"].replace(np.nan, pd.NaT)
            df = df.astype({"data_hora_registro_occ": "datetime64[ns]"})

            # Calcula a diferença absoluta entre data_hora_registro_occ e data_hora_registro e data_hora_final
            df["diff_registro"] = np.where(
                mask,
                (df["data_hora_registro_occ"] - df["data_hora_registro"]).dt.total_seconds().abs(),
                np.nan,
            )
            df["diff_final"] = np.where(
                mask,
                (df["data_hora_registro_occ"] - df["data_hora_final"]).dt.total_seconds().abs(),
                np.nan,
            )

            # Encontra qual das duas diferenças é menor
            mask_idxmin = df[["diff_registro", "diff_final"]].notna().any(axis=1)
            df.loc[mask & mask_idxmin, "closest"] = df.loc[
                mask & mask_idxmin, ["diff_registro", "diff_final"]
            ].idxmin(
                axis=1
            )  # removendo erro:
            # FutureWarning: The behavior of DataFrame.idxmin with all-NA values,
            # or any-NA and skipna=False, is deprecated.
            # In a future version this will raise ValueError

            columns = [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro_occ",
                "usuario_id_occ",
            ]

            # Move a ocorrência para a linha anterior se a data_hora_registro for mais próxima
            mask = (
                (df["closest"].shift(-1) == "diff_registro")
                & (df["status"].shift(-1) == "rodando")
                & df["motivo_id"].isnull()
            )
            for column in columns:
                df[column] = np.where(mask, df[column].shift(-1), df[column])
                df[column] = np.where(mask.shift(1), pd.NaT, df[column])

            # Move a ocorrência para a linha seguinte se a data_hora_final for mais próxima
            mask = (
                (df["closest"].shift(1) == "diff_final")
                & (df["status"].shift(1) == "rodando")
                & df["motivo_id"].isnull()
            )
            for column in columns:
                df[column] = np.where(mask, df[column].shift(1), df[column])
                df[column] = np.where(mask.shift(-1), pd.NaT, df[column])

            return df

        df_info = move_columns(df_info)

        # Remove colunas desnecessárias
        df_info.drop(
            columns=[
                "diff_registro",
                "diff_final",
                "closest",
            ],
            inplace=True,
        )

        # Ajustar motivo id
        df_info["motivo_id"] = df_info["motivo_id"].fillna(np.nan).round(0).astype(float)

        # Ajustar em problema, solucao, usuario_id_occ e motivo_nome
        df_info["problema"] = df_info["problema"].fillna("")
        df_info["solucao"] = df_info["solucao"].fillna("")
        df_info["usuario_id_occ"] = np.where(
            df_info["usuario_id_occ"] == pd.NaT, np.nan, df_info["usuario_id_occ"]
        )
        df_info["motivo_nome"] = np.where(
            df_info["motivo_nome"] == pd.NaT, np.nan, df_info["motivo_nome"]
        )
        # Mudar de np.nan para pd.NaT em data_hora_registro_occ
        df_info["data_hora_registro_occ"] = np.where(
            df_info["data_hora_registro_occ"].isnull(), pd.NaT, df_info["data_hora_registro_occ"]
        )

        # Corrigir os formatos das colunas
        df_info = df_info.astype(
            {
                "data_hora_registro_occ": "datetime64[ns]",
                "motivo_nome": "category",
                "problema": str,
                "solucao": str,
                "usuario_id_occ": "category",
            }
        )

        # Ajustar problema para Domingo, Sábado e Feriado se motivo_id for 12
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["domingo"] == 1),
            "Domingo",
            df_info["problema"],
        )
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["sabado"] == 1),
            "Sábado",
            df_info["problema"],
        )
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["feriado"] == 1),
            "Feriado",
            df_info["problema"],
        )

        # Se o motivo_id for nulo, status for parada e o tempo registro for maior que 475 e for sábado, domingo ou feriado, então o problema é "Domingo", "Sábado" ou "Feriado" e o motivo id é 12 e o motivo nome é "Parada Programada"
        condition = (
            (df_info["motivo_id"].isnull())
            & (df_info["status"] == "parada")
            & (df_info["tempo_registro_min"] > 475)
        )
        condition_sabado = condition & (df_info["sabado"] == 1)
        df_info["motivo_id"] = np.where(condition_sabado, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_sabado, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_sabado, "Sábado", df_info["problema"])

        condition_domingo = condition & (df_info["domingo"] == 1)
        df_info["motivo_id"] = np.where(condition_domingo, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_domingo, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_domingo, "Domingo", df_info["problema"])

        condition_feriado = condition & (df_info["feriado"] == 1)
        df_info["motivo_id"] = np.where(condition_feriado, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_feriado, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_feriado, "Feriado", df_info["problema"])

        # Cria a máscara para as linhas que atendem às condições
        mask = (df_info["tempo_registro_min"] > 475) & df_info["motivo_id"].isnull()

        # Cria colunas temporárias com os valores preenchidos
        df_info["motivo_id_filled"] = df_info["motivo_id"].ffill()
        df_info["motivo_nome_filled"] = df_info["motivo_nome"].ffill()
        df_info["problema_filled"] = df_info["problema"].ffill()
        df_info["solucao_filled"] = df_info["solucao"].ffill()

        # Aplica a máscara e substitui os valores nas colunas originais
        df_info.loc[mask, "motivo_id"] = df_info.loc[mask, "motivo_id_filled"]
        df_info.loc[mask, "motivo_nome"] = df_info.loc[mask, "motivo_nome_filled"]
        df_info.loc[mask, "problema"] = df_info.loc[mask, "problema_filled"]
        df_info.loc[mask, "solucao"] = df_info.loc[mask, "solucao_filled"]

        # Remove as colunas temporárias
        df_info = df_info.drop(
            columns=["motivo_id_filled", "motivo_nome_filled", "problema_filled", "solucao_filled"]
        )

        # Se o tempo de registro for maior que 480 mudar para 480
        df_info["tempo_registro_min"] = np.where(
            df_info["tempo_registro_min"] > 480, 480, df_info["tempo_registro_min"]
        )

        # Remover as linhas onde a linha é 0
        df_info = df_info[df_info["linha"] != 0]

        # Remover as linhas onde status é rodando
        df_info = df_info[df_info["status"] != "rodando"]

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def problems_adjust(self, df: pd.DataFrame, threshold=88) -> pd.DataFrame:
        """
        Ajusta os problemas no DataFrame fornecido, mapeando problemas semelhantes para um nome comum.

        Args:
            df (pd.DataFrame): O DataFrame contendo os problemas a serem ajustados.
            threshold (int, opcional): O limite de similaridade para combinar problemas. Por padrão é 88.

        Returns:
            pd.DataFrame: O DataFrame com problemas ajustados.
        """
        # Encontrar problemas únicos
        unique_problems = df["problema"].unique()
        problem_mapping = {}

        # Criar um dicionário para mapear os problemas
        for problem in unique_problems:
            if problem and problem not in problem_mapping:
                problem = str(problem).capitalize()

                # Corrigir erros básicos de digitação
                problem = problem.replace("Beckup", "Backup")
                problem = problem.replace("Becukp", "Backup")
                problem = problem.replace("Stm", "Atm")

                matches = process.extract(problem, unique_problems, limit=len(unique_problems))

                # Encontrar os problemas com maior similaridade
                similar_problems = [match[0] for match in matches if match[1] >= threshold]

                # Criar um dicionário com os problemas similares
                for similar_problem in similar_problems:
                    problem_mapping[similar_problem] = problem

        # Mapear os problemas
        df["problema"] = df["problema"].map(problem_mapping)

        return df


join_data = JoinData()

In [20]:
df_info_occ = join_data.join_info_occ(df_maq_occ_cleaned.copy(), df_info_cleaned.copy())
df_maq_info_occ_combined = join_data.problems_adjust(df_info_occ.copy())
df_maq_info_occ_combined

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,motivo_nome,problema,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,TMF001,8,1,NOT,2024-01-01 00:00:21,480,2024-01-01 08:02:22,parada,12.0,NaT,Parada Programada,Feriado,NaT,NaN,0,0,0,0,1
1,TMF001,8,1,MAT,2024-01-01 08:02:22,480,2024-01-01 16:02:24,parada,12.0,NaT,Parada Programada,Feriado,NaT,NaN,0,0,0,0,1
2,TMF001,8,1,VES,2024-01-01 16:02:24,480,2024-01-02 00:02:25,parada,12.0,NaT,Parada Programada,Feriado,NaT,NaN,0,0,0,0,1
3,TMF001,8,1,NOT,2024-01-02 00:02:25,480,2024-01-02 08:02:27,parada,12.0,NaT,Parada Programada,NaN,NaT,NaN,0,0,0,0,0
4,TMF001,8,1,MAT,2024-01-02 08:02:27,480,2024-01-02 16:02:28,parada,12.0,NaT,Parada Programada,NaN,NaT,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4165,TMF015,3,1,VES,2024-01-29 17:27:22,8,2024-01-29 17:35:22,parada,NaN,NaT,NaN,NaN,NaT,NaN,724,724,0,0,0
4166,TMF015,3,1,VES,2024-01-29 22:25:23,16,2024-01-29 22:41:23,parada,NaN,NaT,NaN,NaN,NaT,NaN,4114,4106,0,0,0
4167,TMF015,3,1,VES,2024-01-29 23:17:23,14,2024-01-29 23:31:23,parada,NaN,NaT,NaN,NaN,NaT,NaN,4974,4966,0,0,0
4168,TMF015,3,1,VES,2024-01-29 23:59:23,2,2024-01-30 00:01:23,parada,NaN,NaT,NaN,NaN,NaT,NaN,5360,5352,0,0,0


In [21]:
df_maq_info_occ_combined.columns

Index(['maquina_id', 'linha', 'fabrica', 'turno', 'data_hora_registro',
       'tempo_registro_min', 'data_hora_final', 'status', 'motivo_id',
       'data_hora_registro_occ', 'motivo_nome', 'problema', 'solucao',
       'usuario_id_occ', 'contagem_total_ciclos', 'contagem_total_produzido',
       'sabado', 'domingo', 'feriado'],
      dtype='object')

In [23]:
df_maq_info_production_cleaned.columns

Index(['maquina_id', 'linha', 'turno', 'total_ciclos', 'total_produzido',
       'data_registro'],
      dtype='object')

# Eficiência, Performance, Reparos


In [40]:
# service/times_data.py
# cSpell: disable=invalid-name
class TimesData:
    def __init__(self):
        # Dicionário com os descontos de parada para Eficiência

        self.desc_eff = {
            3: 60,
            5: 10,
            10: 15,
            11: 35,
            15: 60,
            17: 15,
        }

        # Dicionário com os descontos de parada para Performance

        self.desc_perf = {
            3: 60,
            5: 10,
            10: 15,
            15: 60,
            17: 15,
        }

        # Lista com os motivos de parada que não são considerados para Performance

        self.not_af_perf = [7, 8, 11, 12, 13, 16]

        # Dicionário com os descontos de parada para Reparos

        self.desc_rep = {11: 35}

        # Lista com os motivos de parada que são considerados para Reparos

        self.af_rep = [7, 8, 11]

    def get_times_discount(self, info: pd.DataFrame, desc_pcp: dict[int, int]) -> pd.DataFrame:
        """
        Função para calcular os descontos de parada

        Args:
            info (pd.DataFrame): DataFrame com os dados de parada
            desc_pcp (dict[int, int]): Dicionário com os descontos de parada

        Returns:
            pd.DataFrame: DataFrame com os descontos de parada


        Exemplo:
            >>> from app.service.get_times_data import GetTimesData
            >>> import pandas as pd
            >>> get_times_data = GetTimesData()
            >>> df_times_desc = pd.dataframe()
            >>> df_result = get_times_data.get_times_discount(df_times_desc, desc_pcp)
        """

        info_stops = info.copy()

        # Adicionar coluna com descontos de parada
        info_stops["desconto_min"] = info_stops["motivo_id"].map(desc_pcp)

        # Se houver desconto, subtrair do tempo de parada e arredondar para baixo, em uma nova coluna chamada excedente
        info_stops["excedente"] = (
            info_stops["tempo_registro_min"] - info_stops["desconto_min"]
        ).clip(lower=0)

        # Se o desconto for maior que o tempo de parada, o desconto deve ser igual ao tempo de parada
        info_stops.loc[
            info_stops["desconto_min"] > info_stops["tempo_registro_min"],
            "desconto_min",
        ] = info_stops["tempo_registro_min"]

        # Criar coluna data_registro para agrupar por dia
        info_stops["data_registro"] = info_stops["data_hora_registro"].dt.date
        # Ordenar por maquina_id, data_hora_registro, turno
        info_stops.sort_values(by=["maquina_id", "data_hora_registro", "turno"], inplace=True)

        return info_stops

    def get_elapsed_time(self, turno):
        """
        Método para calcular o tempo decorrido no turno atual.

        Este método recebe o turno atual e retorna o tempo decorrido em minutos.

        Args:
            turno (str): Turno atual

        Returns:
            float: Tempo decorrido em minutos


        Exemplo:
            >>> from app.service.get_times_data import GetTimesData
            >>> import pandas as pd
            >>> get_times_data = GetTimesData()
            >>> turno = 'MAT'
            >>> tempo_decorrido = get_times_data.get_elapsed_time(turno)
        """

        now = datetime.now()

        if turno == "MAT" and 8 <= now.hour < 16:
            shift_start = now.replace(hour=8, minute=0, second=0, microsecond=0)

        elif turno == "VES" and 16 <= now.hour < 24:
            shift_start = now.replace(hour=16, minute=0, second=0, microsecond=0)

        elif turno == "NOT" and (now.hour < 8 or now.hour >= 24):
            shift_start = now.replace(hour=0, minute=0, second=0, microsecond=0)

        else:
            return 480  # retorna o tempo padrão se não estiver no turno atual

        elapsed_time = now - shift_start

        return elapsed_time.total_seconds() / 60  # retorna o tempo decorrido em minutos

    def get_eff_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de eficiência.
        Este método recebe dois DataFrames, um contendo informações de tempo de eficiência e
        desconto e outro contendo informações de produção,
        e retorna um DataFrame com informações de eficiência.


        Parâmetros:

        df_info (pd.DataFrame): DataFrame contendo informações de maquina
        df_prod (pd.DataFrame): DataFrame contendo informações de produção.


        Retorna:

        pd.DataFrame: DataFrame com informações de eficiência.


        Exemplo de uso:
        ```
        times_data = TimesData()
        df_eff_times_desc = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_eff_data(df_info, df_prod)
        ```
        """

        df_eff_times_desc = self.get_times_discount(df_info, self.desc_eff)
        df_prod_total = df_prod.copy()
        ciclo_ideal = 10.6

        # Descartar colunas desnecessárias de df_prod -> 'contagem_total_ciclos', 'usuario_id_maq_cadastro', 'data_hora_registro'
        df_prod_total.drop(columns=["total_ciclos"], inplace=True)

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de parada, o desconto e o excedente
        df_eff_times_desc = (
            df_eff_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_eff_times_desc["data_registro"] = pd.to_datetime(
            df_eff_times_desc["data_registro"]
        ).dt.date

        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date

        # Fazer merge com df_prod_total
        df_eff_times_desc = pd.merge(
            df_prod_total,
            df_eff_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desc_min para 0 quando for nulo
        df_eff_times_desc.loc[df_eff_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Criar coluna com tempo esperado de produção
        df_eff_times_desc["tempo_esperado_min"] = df_eff_times_desc.apply(
            lambda row: np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
            if row["data_registro"] == datetime.now().date()
            else 480 - row["desconto_min"],
            axis=1,
        )

        # Produção esperada por turno
        df_eff_times_desc["producao_esperada"] = (
            df_eff_times_desc["tempo_esperado_min"] * ciclo_ideal
        ) * 2

        # Calcular a eficiência
        df_eff_times_desc["eficiencia"] = (
            df_eff_times_desc["total_produzido"] / df_eff_times_desc["producao_esperada"]
        )

        # Ordenar pela linha e data_registro
        df_eff_times_desc = df_eff_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_eff_times_desc = df_eff_times_desc[df_eff_times_desc["linha"] != 0]

        # Ajustar o index
        df_eff_times_desc.reset_index(drop=True, inplace=True)

        return df_eff_times_desc

    def get_perf_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de performance.

        Este método recebe dois DataFrames, um contendo informações de máquina e
        outro contendo informações de produção,
        e retorna um DataFrame com informações de performance.

        Args:
            df_info (pd.DataFrame): DataFrame contendo informações de maquina
            df_prod (pd.DataFrame): DataFrame contendo informações de produção.

        Returns:
            pd.DataFrame: DataFrame com informações de performance.

        Exemplo de uso:
        ```
        times_data = TimesData()
        df_info = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_perf_data(df_info, df_prod)
        ```
        """

        df_info = df_info.copy()
        df_perf_times_desc = self.get_times_discount(df_info, self.desc_perf)
        df_prod_total = df_prod.copy()

        # Descartar colunas desnecessárias de df_prod
        df_prod_total.drop(
            columns=[
                "total_ciclos",
                "total_produzido",
            ],
            inplace=True,
        )

        # Remover as linhas que não afetam a performance
        df_perf_times_desc = df_perf_times_desc[
            ~df_perf_times_desc["motivo_id"].isin(self.not_af_perf)
        ]

        # Criar coluna 'afeta' para identificar as paradas que afetam a performance
        df_perf_times_desc["afeta"] = df_perf_times_desc["excedente"]

        # Se desconto for nulo, substituir afeta pelo valor de tempo_registro_min
        df_perf_times_desc.loc[
            df_perf_times_desc["desconto_min"].isnull(), "afeta"
        ] = df_perf_times_desc["tempo_registro_min"]

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de
        # desconto e o afeta
        df_perf_times_desc = (
            df_perf_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                    "afeta": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_perf_times_desc["data_registro"] = pd.to_datetime(
            df_perf_times_desc["data_registro"]
        ).dt.date

        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date
        # Fazer merge com df_prod_total

        df_perf_times_desc = pd.merge(
            df_prod_total,
            df_perf_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desconto_min para 0 quando for nulo
        df_perf_times_desc.loc[df_perf_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Ajustar afeta para 0 quando for nulo
        df_perf_times_desc.loc[df_perf_times_desc["afeta"].isnull(), "afeta"] = 0

        # Criar coluna com tempo esperado de produção
        df_perf_times_desc["tempo_esperado_min"] = df_perf_times_desc.apply(
            lambda row: np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
            if row["data_registro"] == datetime.now().date()
            else 480 - row["desconto_min"],
            axis=1,
        )

        # Calcular a performance
        df_perf_times_desc["performance"] = (
            df_perf_times_desc["afeta"] / df_perf_times_desc["tempo_esperado_min"]
        )

        # Ordenar pela linha e data_registro
        df_perf_times_desc = df_perf_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_perf_times_desc = df_perf_times_desc[df_perf_times_desc["linha"] != 0]

        # Ajustar o index
        df_perf_times_desc.reset_index(drop=True, inplace=True)

        return df_perf_times_desc

    def get_reparos_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de reparo.

        Este método recebe dois DataFrames, um contendo informações de máquina e
        e outro contendo informações de produção,
        e retorna um DataFrame com informações de Reparo.


        ### Parâmetros:
        df_info (pd.DataFrame): DataFrame contendo informações de maquina

        df_prod (pd.DataFrame): DataFrame contendo informações de produção.

        ### Retorna:

        pd.DataFrame: DataFrame com informações de performance.

        ### Exemplo de uso:
        ```
        times_data = TimesData()
        df_info = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_reparos_data(df_info, df_prod)
        ```
        """

        df_info = df_info.copy()

        df_rep_times_desc = self.get_times_discount(df_info, self.desc_rep)

        df_prod_total = df_prod.copy()

        # Descartar colunas desnecessárias de df_prod
        df_prod_total.drop(
            columns=[
                "total_ciclos",
                "total_produzido",
            ],
            inplace=True,
        )

        # Remover as linhas que não afetam o reparo
        df_rep_times_desc = df_rep_times_desc[df_rep_times_desc["motivo_id"].isin(self.af_rep)]

        # Criar coluna 'afeta' para identificar as paradas que afetam o reparo
        df_rep_times_desc["afeta"] = df_rep_times_desc["excedente"]

        # Se desconto for nulo, substituir afeta pelo valor de tempo_registro_min
        df_rep_times_desc.loc[
            df_rep_times_desc["desconto_min"].isnull(), "afeta"
        ] = df_rep_times_desc["tempo_registro_min"]

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de
        # desconto e o afeta
        df_rep_times_desc = (
            df_rep_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                    "afeta": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_rep_times_desc["data_registro"] = pd.to_datetime(
            df_rep_times_desc["data_registro"]
        ).dt.date
        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date

        # Fazer merge com df_prod_total
        df_rep_times_desc = pd.merge(
            df_prod_total,
            df_rep_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desconto_min para 0 quando for nulo
        df_rep_times_desc.loc[df_rep_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Ajustar afeta para 0 quando for nulo
        df_rep_times_desc.loc[df_rep_times_desc["afeta"].isnull(), "afeta"] = 0

        # Criar coluna com tempo esperado de produção
        df_rep_times_desc["tempo_esperado_min"] = df_rep_times_desc.apply(
            lambda row: np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
            if row["data_registro"] == datetime.now().date()
            else 480 - row["desconto_min"],
            axis=1,
        )

        # Calcular o reparo
        df_rep_times_desc["reparo"] = (
            df_rep_times_desc["afeta"] / df_rep_times_desc["tempo_esperado_min"]
        )

        # Ordenar pela linha e data_registro
        df_rep_times_desc = df_rep_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_rep_times_desc = df_rep_times_desc[df_rep_times_desc["linha"] != 0]

        # Ajustar o index
        df_rep_times_desc.reset_index(drop=True, inplace=True)

        return df_rep_times_desc


times_data = TimesData()

In [41]:
df_eff = times_data.get_eff_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_eff

,maquina_id,linha,turno,total_produzido,data_registro,desconto_min,tempo_esperado_min,producao_esperada,eficiencia
0,TMF005,1,NOT,0,2024-01-01,0.0,480.0,10176.0,0.000000
1,TMF005,1,MAT,0,2024-01-01,0.0,480.0,10176.0,0.000000
2,TMF005,1,VES,0,2024-01-01,0.0,480.0,10176.0,0.000000
3,TMF005,1,NOT,0,2024-01-02,0.0,480.0,10176.0,0.000000
4,TMF005,1,MAT,0,2024-01-02,0.0,480.0,10176.0,0.000000
...,...,...,...,...,...,...,...,...,...
1183,TMF010,14,VES,3038,2024-01-28,0.0,480.0,10176.0,0.298546
1184,TMF010,14,NOT,4010,2024-01-29,0.0,480.0,10176.0,0.394064
1185,TMF010,14,MAT,2188,2024-01-29,60.0,420.0,8904.0,0.245732
1186,TMF010,14,VES,2768,2024-01-29,60.0,420.0,8904.0,0.310872


In [42]:
df_perf = times_data.get_perf_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_perf

,maquina_id,linha,turno,data_registro,desconto_min,afeta,tempo_esperado_min,performance
0,TMF005,1,NOT,2024-01-01,0.0,0.0,480.0,0.000000
1,TMF005,1,MAT,2024-01-01,0.0,0.0,480.0,0.000000
2,TMF005,1,VES,2024-01-01,0.0,0.0,480.0,0.000000
3,TMF005,1,NOT,2024-01-02,0.0,0.0,480.0,0.000000
4,TMF005,1,MAT,2024-01-02,0.0,0.0,480.0,0.000000
...,...,...,...,...,...,...,...,...
1183,TMF010,14,VES,2024-01-28,0.0,0.0,480.0,0.000000
1184,TMF010,14,NOT,2024-01-29,0.0,0.0,480.0,0.000000
1185,TMF010,14,MAT,2024-01-29,60.0,26.0,420.0,0.061905
1186,TMF010,14,VES,2024-01-29,60.0,70.0,420.0,0.166667


In [43]:
df_reparos = times_data.get_reparos_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_reparos

,maquina_id,linha,turno,data_registro,desconto_min,afeta,tempo_esperado_min,reparo
0,TMF005,1,NOT,2024-01-01,0.0,0.0,480.0,0.000000
1,TMF005,1,MAT,2024-01-01,0.0,0.0,480.0,0.000000
2,TMF005,1,VES,2024-01-01,0.0,0.0,480.0,0.000000
3,TMF005,1,NOT,2024-01-02,0.0,0.0,480.0,0.000000
4,TMF005,1,MAT,2024-01-02,0.0,0.0,480.0,0.000000
...,...,...,...,...,...,...,...,...
1183,TMF010,14,VES,2024-01-28,0.0,0.0,480.0,0.000000
1184,TMF010,14,NOT,2024-01-29,0.0,0.0,480.0,0.000000
1185,TMF010,14,MAT,2024-01-29,0.0,46.0,480.0,0.095833
1186,TMF010,14,VES,2024-01-29,0.0,0.0,480.0,0.000000


# Gráficos


In [62]:
# cSpell: disable=invalid-name
def test(dataframe, meta):
    # Converter 'data_registro' para datetime e criar uma nova coluna 'data_turno'
    dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])
    dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

    # Agrupar por 'data_turno' e 'turno' e calcular a média da eficiência
    df_grouped = (
        dataframe.groupby(["data_turno", "turno"], observed=False)["eficiencia"]
        .mean()
        .reset_index()
    )

    # Remodelar os dados para o formato de heatmap
    df_pivot = df_grouped.pivot(index="turno", columns="data_turno", values="eficiencia")

    # Reordenar o índice do DataFrame
    df_pivot = df_pivot.reindex(["VES", "MAT", "NOT"])

    # Criar escala de cores personalizada - cores do bootstrap
    colors = [
        [0, "#e30613"],
        [0.9, "#e30613"],
        [0.9, "#00a13a"],
        [1, "#00a13a"],
    ]

    # Extrair apenas o dia da data
    df_pivot.columns = pd.to_datetime(df_pivot.columns).day

    # Criar o gráfico de calor
    fig = go.Figure(
        data=go.Heatmap(
            z=df_pivot.values,
            x=df_pivot.columns,
            y=df_pivot.index,
            colorscale=colors,
            zmin=0,
            zmax=1,  # Escala de valores de 0 a 1
            hoverongaps=False,
            hovertemplate="Turno: %{y}<br>Dia: %{x}<br>Valor: %{z:.1%}",
            showscale=False,  # Não mostrar a escala de cores
            xgap=1,  # Espaçamento entre os dias
            ygap=1,  # Espaçamento entre os turnos
        )
    )

    # Adicionar anotações com a média da eficiência
    for (i, j), value in np.ndenumerate(df_pivot.values):
        fig.add_annotation(
            x=df_pivot.columns[j],
            y=df_pivot.index[i],
            text=f"{value:.1%}",
            showarrow=False,
            font=dict(color="white", size=8),
        )

    # Definir o título do gráfico
    fig.update_layout(
        title=f"Eficiência - Meta {meta}%",
        xaxis_title="Dia",
        yaxis_title="Turno",
        title_x=0.5,  # Centralizar o título
        xaxis_nticks=31,  # Definir o número de dias
        xaxis=dict(
            tickmode="linear",
            tickvals=list(range(1, 32)),  # Definir os dias
            ticktext=list(range(1, 32)),  # Definir os dias
            tickangle=45,  # Rotacionar os dias
        ),
        yaxis=dict(
            tickmode="linear",
            tickangle=45,
        ),
        plot_bgcolor="white",
        margin=dict(t=40, b=40, l=40, r=40),
        font=dict({"family": "Inter"}),
    )

    fig.show()


test(df_eff, 85)

In [69]:
# cSpell: disable=invalid-name
def graph_heatmap_perf_v2(dataframe: pd.DataFrame, meta: int):
    # Converter 'data_registro' para datetime e criar uma nova coluna 'data_turno'

    dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])

    dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

    # Agrupar por 'data_turno' e 'turno' e calcular a média da eficiência

    df_grouped = (
        dataframe.groupby(["data_turno", "turno"], observed=False)["performance"]
        .mean()
        .reset_index()
    )

    # Remodelar os dados para o formato de heatmap

    df_pivot = df_grouped.pivot(index="turno", columns="data_turno", values="performance")

    # Reordenar o índice do DataFrame
    df_pivot = df_pivot.reindex(["VES", "MAT", "NOT"])

    # Criar escala de cores personalizada
    colors = [[0, "green"], [0.04, "green"], [0.04, "red"], [1, "red"]]

    # Extrair apenas o dia da data
    df_pivot.columns = pd.to_datetime(df_pivot.columns).day

    # Criar o gráfico de calor
    fig = go.Figure(
        data=go.Heatmap(
            z=df_pivot.values,
            x=df_pivot.columns,
            y=df_pivot.index,
            colorscale=colors,
            zmin=0,
            zmax=1,  # Escala de valores de 0 a 1
            hoverongaps=False,
            hovertemplate="Turno: %{y}<br>Dia: %{x}<br>Valor: %{z:.1%}",
            showscale=False,  # Não mostrar a escala de cores
            xgap=1,  # Espaçamento entre os dias
            ygap=1,  # Espaçamento entre os turnos
        )
    )

    # Adicionar anotações com a média da eficiência
    for (i, j), value in np.ndenumerate(df_pivot.values):
        fig.add_annotation(
            x=df_pivot.columns[j],
            y=df_pivot.index[i],
            text=f"{value:.1%}",
            showarrow=False,
            font=dict(color="white", size=8),
        )

    # Definir o título do gráfico
    fig.update_layout(
        title=f"Performance - Meta {meta}%",
        xaxis_title="Dia",
        yaxis_title="Turno",
        title_x=0.5,  # Centralizar o título
        xaxis_nticks=31,  # Definir o número de dias
        xaxis=dict(
            tickmode="linear",
            tickvals=list(range(1, 32)),  # Definir os dias
            ticktext=list(range(1, 32)),  # Definir os dias
            tickangle=45,  # Rotacionar os dias
        ),
        yaxis=dict(
            tickmode="linear",
            tickangle=45,
        ),
        plot_bgcolor="white",
        margin=dict(t=40, b=40, l=40, r=40),
        font=dict({"family": "Inter"}),
    )

    fig.show()


graph_heatmap_perf_v2(df_perf, 4)

In [70]:
# cSpell: disable=invalid-name
def graph_heatmap_reparos(dataframe: pd.DataFrame, meta: int):
    # Converter 'data_registro' para datetime e criar uma nova coluna 'data_turno'

    dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])

    dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

    # Agrupar por 'data_turno' e 'turno' e calcular a média da eficiência

    df_grouped = (
        dataframe.groupby(["data_turno", "turno"], observed=False)["reparo"].mean().reset_index()
    )

    # Remodelar os dados para o formato de heatmap

    df_pivot = df_grouped.pivot(index="turno", columns="data_turno", values="reparo")

    # Reordenar o índice do DataFrame
    df_pivot = df_pivot.reindex(["VES", "MAT", "NOT"])

    # Criar escala de cores personalizada
    colors = [[0, "green"], [0.04, "green"], [0.04, "red"], [1, "red"]]

    # Extrair apenas o dia da data
    df_pivot.columns = pd.to_datetime(df_pivot.columns).day

    # Criar o gráfico de calor
    fig = go.Figure(
        data=go.Heatmap(
            z=df_pivot.values,
            x=df_pivot.columns,
            y=df_pivot.index,
            colorscale=colors,
            zmin=0,
            zmax=1,  # Escala de valores de 0 a 1
            hoverongaps=False,
            hovertemplate="Turno: %{y}<br>Dia: %{x}<br>Valor: %{z:.1%}",
            showscale=False,  # Não mostrar a escala de cores
            xgap=1,  # Espaçamento entre os dias
            ygap=1,  # Espaçamento entre os turnos
        )
    )

    # Adicionar anotações com a média da eficiência
    for (i, j), value in np.ndenumerate(df_pivot.values):
        fig.add_annotation(
            x=df_pivot.columns[j],
            y=df_pivot.index[i],
            text=f"{value:.1%}",
            showarrow=False,
            font=dict(color="white", size=8),
        )

    # Definir o título do gráfico
    fig.update_layout(
        title=f"Reparos - Meta {meta}%",
        xaxis_title="Dia",
        yaxis_title="Turno",
        title_x=0.5,  # Centralizar o título
        xaxis_nticks=31,  # Definir o número de dias
        xaxis=dict(
            tickmode="linear",
            tickvals=list(range(1, 32)),  # Definir os dias
            ticktext=list(range(1, 32)),  # Definir os dias
            tickangle=45,  # Rotacionar os dias
        ),
        yaxis=dict(
            tickmode="linear",
            tickangle=45,
        ),
        plot_bgcolor="white",
        margin=dict(t=40, b=40, l=40, r=40),
    )

    fig.show()


reparos_map = graph_heatmap_reparos(df_reparos, 4)

In [71]:
# cSpell: disable=invalid-name


def calculate_eficiencia(df):
    return df["eficiencia"].mean()


def calculate_performance(df):
    return df["performance"].mean()


def calculate_reparos(df):
    return df["reparo"].mean()


def draw_circular_progress_bar_v2(df: pd.DataFrame, type: str, meta: int):
    # mapear o tipo das funções
    type_map = {
        "eficiencia": calculate_eficiencia,
        "performance": calculate_performance,
        "reparo": calculate_reparos,
    }

    # Verificar a primeira data_registro do dataframe para saber se os dados são do mês atual
    this_month = df["data_registro"].iloc[0].month == pd.Timestamp.now().month
    month = "Atual" if this_month else "Anterior"

    # Obter a função com base no tipo
    func = type_map[type]

    # Calcular a porcentagem
    percentage = func(df.copy()) if func is not None else 0

    # Arredondar porcentagem para não ter casas decimais
    percentage = round(percentage, 2)

    # Definir a cor com base na porcentagem
    if type == "eficiencia":
        color = "green" if percentage >= (meta / 100) else "red"
    else:
        color = "red" if percentage >= (meta / 100) else "green"

    # Definir a escala do eixo para "performance" e "reparo"
    axis_range = [0, 100] if type == "eficiencia" else [100, 0]

    # Criar o gráfico
    fig = go.Figure(
        go.Indicator(
            mode="gauge+number",
            value=percentage * 100,
            number={"suffix": "%"},
            domain={"x": [0, 1], "y": [0, 1]},
            title={
                "text": month,
                "font": {"size": 14},
            },
            gauge={
                "axis": {
                    "range": axis_range,
                    "tickfont": {"size": 8},
                },
                "bar": {"color": color},
                "steps": [
                    {"range": [0, 100], "color": "lightgray"},
                ],
                "threshold": {
                    "line": {"color": "black", "width": 4},
                    "thickness": 0.75,
                    "value": meta,
                },
            },
        )
    )

    fig.update_layout(
        autosize=True,
        margin=dict(t=30, b=30, l=30, r=30),
        plot_bgcolor="white",
        width=200,  # apenas para visualização no jupyter
        height=200,  # apenas para visualização no jupyter
    )

    fig.show()


draw_circular_progress_bar_v2(df_eff, "eficiencia", 90)
draw_circular_progress_bar_v2(df_perf, "performance", 4)
draw_circular_progress_bar_v2(df_reparos, "reparo", 4)

In [72]:
# cSpell: disable=invalid-name
def line_graph(df: pd.DataFrame, indicator: str):
    # Converter 'data_registro' para datetime e criar uma nova coluna 'data_turno'

    df["data_registro"] = pd.to_datetime(df["data_registro"])

    df["data_turno"] = df["data_registro"].dt.strftime("%Y-%m-%d")

    # Agrupar por 'data_turno' e 'turno' e calcular a média da eficiência

    df_grouped = df.groupby(["data_turno"])[indicator].mean().reset_index()

    # Multiplicar a eficiência por 100 para converter para porcentagem

    df_grouped[indicator] = df_grouped[indicator] * 100

    # Criar o gráfico

    fig = go.Figure(
        go.Scatter(
            x=df_grouped["data_turno"],
            y=df_grouped[indicator],
            mode="lines+markers",
            line=dict(color="blue"),
            marker=dict(color="blue"),
            hovertemplate="<i>Dia</i>: %{x}" + "<br><b>Porcentagem</b>: %{y:.1f}<br>",
            hoverinfo="skip",
        )
    )

    fig.update_layout(
        showlegend=False,
        plot_bgcolor="white",
        xaxis=dict(showticklabels=False),  # Esconde os valores do eixo x
        yaxis=dict(showticklabels=False),
        margin=dict(t=0, b=0, l=0, r=0),
        # height=None, # removido apenas para visualização no jupyter
        # autosize=True, # removido apenas para visualização no jupyter
        height=200,  # apenas para visualização no jupyter
    )

    fig.show()


line_graph(df_eff, "eficiencia")


line_graph(df_perf, "performance")


line_graph(df_reparos, "reparo")

In [80]:
# cSpell: disable=invalid-name
import matplotlib.colors as mcolors
import seaborn as sns


class IndicatorsTurn:
    """
    Esta classe é responsável por criar os gráficos de indicadores.
    """

    def __init__(self):
        self.danger_color = "#dc3545"
        self.warning_color = "#ffc107"
        self.success_color = "#198754"

    def get_eff_heat_turn(
        self,
        dataframe: pd.DataFrame,
        meta: int = 90,
        annotations: bool = False,
    ) -> go.Figure:
        """
        Este método é responsável por criar o gráfico de eficiência, por turno.

        Parâmetros:
        dataframe (pd.DataFrame): DataFrame contendo os dados para o gráfico.
                                Deve incluir as colunas 'data_registro', 'turno' e 'eficiencia'.
        meta (int): Meta de eficiência a ser alcançada. Padrão: 90.
        annotations (bool): Se True, adiciona anotações com a média da eficiência.

        Retorna:
        fig: Objeto plotly.graph_objects.Figure com o gráfico de eficiência.

        O gráfico é um heatmap que mostra a eficiência média por maquina e data.
        A eficiência é colorida de vermelho se estiver abaixo de 90% e
        de verde se estiver acima de 90%.
        """

        # Converter 'data_registro' para datetime e criar uma nova coluna 'data_turno'
        dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])
        dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

        # Agrupar por 'data_turno' e 'turno' e calcular a média da eficiência
        df_grouped = (
            dataframe.groupby(["data_turno", "linha"], observed=False)["eficiencia"]
            .mean()
            .reset_index()
        )

        # Ordenar por linha e data
        df_grouped = df_grouped.sort_values(["linha", "data_turno"], ascending=[True, True])

        # Remodelar os dados para o formato de heatmap
        df_pivot = df_grouped.pivot(index="linha", columns="data_turno", values="eficiencia")

        # Criar escala de cores personalizada - cores do bootstrap
        colors = [
            [0, self.danger_color],
            [0.9, self.danger_color],
            [0.9, self.success_color],
            [1, self.success_color],
        ]

        # Extrair apenas o dia da data
        df_pivot.columns = pd.to_datetime(df_pivot.columns).day

        num_cells = len(df_pivot.index) * len(df_pivot.columns)
        font_size = 3000 / num_cells

        # Criar o gráfico de calor
        fig = go.Figure(
            data=go.Heatmap(
                z=df_pivot.values,
                x=df_pivot.columns,
                y=df_pivot.index,
                colorscale=colors,
                zmin=0,
                zmax=1,  # Escala de valores de 0 a 1
                hoverongaps=False,
                hovertemplate="Linha: %{y}<br>Dia: %{x}<br>Eficiência: %{z:.1%}",
                showscale=False,  # Não mostrar a escala de cores
                xgap=1,  # Espaçamento entre os dias
                ygap=1,  # Espaçamento entre os turnos
            )
        )

        # Adicionar anotações com a média da eficiência
        if annotations:
            for (i, j), value in np.ndenumerate(df_pivot.values):
                fig.add_annotation(
                    x=df_pivot.columns[j],
                    y=df_pivot.index[i],
                    text=f"{value:.1%}",
                    showarrow=False,
                    font=dict(color="white", size=font_size),
                )

        # Definir o título do gráfico
        fig.update_layout(
            title=f"Eficiência - Meta {meta}%",
            xaxis_title="Dia",
            yaxis_title="Linha",
            title_x=0.5,  # Centralizar o título
            xaxis_nticks=31,  # Definir o número de dias
            xaxis=dict(
                tickmode="linear",
                tickvals=list(range(1, 32)),  # Definir os dias
                ticktext=list(range(1, 32)),  # Definir os dias
                tickangle=45,  # Rotacionar os dias
            ),
            yaxis=dict(
                tickmode="linear",
                tickangle=45,
            ),
            plot_bgcolor="white",
            margin=dict(t=40, b=40, l=40, r=40),
        )

        fig.show()

        return fig

    def get_eff_bar_turn(self, dataframe: pd.DataFrame, meta: int = 90) -> go.Figure:
        # Agrupar por 'turno' e "linha" e calcular a média da eficiência e soma da produção
        df_grouped = (
            dataframe.groupby(["linha", "turno"], observed=False)
            .agg({"eficiencia": "mean", "total_produzido": "sum"})
            .reset_index()
        )

        # Ajustar produção total para caixas, dividindo por 10
        df_grouped["total_produzido"] = (df_grouped["total_produzido"] / 10).round(0)

        # Gráfico de barras
        fig = px.bar(
            df_grouped,
            orientation="h",
            x="eficiencia",
            y="linha",
            color="turno",
            barmode="group",
            hover_data={
                "total_produzido": True,
                "linha": False,
                "eficiencia": False,
            },
            color_discrete_map={
                "NOT": self.danger_color,
                "MAT": self.warning_color,
                "VES": self.success_color,
            },
            labels={"eficiencia": "Eficiência"},
        )

        # Ajustar hover
        fig.update_traces(
            hovertemplate="<b>Linha</b>: %{y}<br><b>Eficiência</b>: %{x:.1%}<br><b>Produção</b>: %{customdata[0]} caixas<br>",
        )

        # Definir o título do gráfico
        fig.update_layout(
            title="Eficiência por Linhas",
            xaxis_title="Eficiência",
            yaxis_title="Linha",
            title_x=0.5,  # Centralizar o título
            plot_bgcolor="white",
            margin=dict(t=40, b=40, l=40, r=40),
            legend=dict(
                title_text="Turno",
            ),
        )

        # Ajustar valores de x para porcentagem
        fig.update_xaxes(tickformat=".0%")

        # Ajustar para aparecer todas as linhas
        fig.update_yaxes(
            autorange="reversed",
            tickvals=df_grouped["linha"].unique(),
        )

        # Calcular a média geral de eficiência
        avg_efficiency = df_grouped["eficiencia"].mean()

        # Adicionar linha de média geral
        fig.add_trace(
            go.Scatter(
                x=[avg_efficiency] * len(df_grouped["linha"]),
                y=df_grouped["linha"],
                mode="lines",
                name="Média Geral",
                line=dict(dash="dash", color="black"),
                hovertemplate="<b>Média Geral</b>: %{x:.1%}<br>",
            )
        )

        # Adicionar linha de meta
        fig.add_trace(
            go.Scatter(
                x=[meta / 100] * len(df_grouped["linha"]),
                y=df_grouped["linha"],
                mode="lines",
                name="Meta",
                line=dict(dash="dash", color="red"),
                hovertemplate="<b>Meta</b>: %{x:.1%}<br>",
            )
        )

        fig.show()

        return fig

    def get_eff_lost(self, df_info: pd.DataFrame) -> pd.DataFrame:
        # Conseguindo dataframe com tempos ajustados
        eff_discount = times_data.desc_eff
        df_info_desc_times = times_data.get_times_discount(df_info, eff_discount)

        # Se coluna "excedente" for nula, substituir pelo valor de "tempo_registro_min"
        df_info_desc_times.loc[
            df_info_desc_times["excedente"].isnull(), "excedente"
        ] = df_info_desc_times["tempo_registro_min"]

        # Se motivo id for nulo e excedente for menor que 15 substituir motivo_nome por
        # "Não apontado - 15min ou menos"
        df_info_desc_times.loc[
            (df_info_desc_times["motivo_id"].isnull()) & (df_info_desc_times["excedente"] <= 15),
            ["motivo_nome", "problema"],
        ] = ["Não apontado - 15min ou menos", "Não apontado - 15min ou menos"]

        # Preencher onde motivo_nome for nulo
        df_info_desc_times["motivo_nome"].fillna("Motivo não informado", inplace=True)

        df_info_desc_times.loc[
            (df_info_desc_times["motivo_id"] == 12)
            & (df_info_desc_times["problema"] == "Parada programada"),
            "problema",
        ] = "Parada Programada"
        return df_info_desc_times

    def get_eff_bar_lost(self, df: pd.DataFrame, turn: str, checked: bool = False) -> go.Figure:
        """
        Retorna um gráfico de barras representando o tempo perdido que mais impacta a eficiência.

        Parâmetros:
        - df: DataFrame contendo os dados necessários para a criação do gráfico.
        - checked: Se True, retorna o gráfico de barras agrupado por motivo_nome e problema.

        Retorno:
        - fig: Objeto go.Figure contendo o gráfico de barras.
        """
        # Turno Map
        turn_map = {
            "NOT": "Noturno",
            "MAT": "Matutino",
            "VES": "Vespertino",
        }

        # ---------- df motivo ---------- #
        # Agrupar motivo_nome
        df_motivo = (
            df.groupby("motivo_nome")["excedente"]
            .sum()
            .sort_values(ascending=False)
            .head(5)
            .reset_index()
        )

        # Preencher onde motivo_id for 3 e problema for nulo
        df.loc[
            (df["motivo_id"] == 3) & (df["problema"].isnull()),
            "problema",
        ] = "Refeição"

        # Preencher onde problema for nulo e motivo_id for 12
        df.loc[
            (df["motivo_id"] == 12) & (df["problema"].isnull()),
            "problema",
        ] = "Parada Programada"

        # Preencher onde problema for nulo
        df.loc[:, "problema"] = df["problema"].fillna("Problema não informado")

        # ---------- df group ---------- #
        # Agrupar por motivo_nome e problema e calcular a soma do excedente
        df_grouped = df.groupby(["motivo_nome", "problema"]).agg({"excedente": "sum"}).reset_index()
        # Ordenar por excedente
        df_grouped = df_grouped.sort_values("excedente", ascending=False).head(8)

        # ---------- df problema ---------- #
        # Remover linhas onde motivo_nome é igual ao problema
        df = df[df["motivo_nome"] != df["problema"]]
        # Agrupar por problema
        df_problema = (
            df.groupby("problema")["excedente"]
            .sum()
            .sort_values(ascending=False)
            .head(5)
            .reset_index()
        )

        # Motivo
        motive_bar = go.Bar(
            name="Motivo",
            x=df_motivo["motivo_nome"],
            y=df_motivo["excedente"],
        )

        motive_bar.update(
            hovertemplate="<b>Motivo</b>: %{x}<br><b>Tempo Perdido</b>: %{y:.0f} min<br>",
        )

        # Problema
        problem_bar = go.Bar(
            name="Problema",
            x=df_problema["problema"],
            y=df_problema["excedente"],
        )

        problem_bar.update(
            hovertemplate="<b>Problema</b>: %{x}<br><b>Tempo Perdido</b>: %{y:.0f} min<br>",
        )

        # Cria uma paleta de cores com os valores únicos na coluna 'problema'
        palette = sns.color_palette("hls", df_grouped["problema"].nunique())

        # Converte as cores RGB para hexadecimal
        palette_hex = [mcolors.to_hex(color) for color in palette]

        # Cria um dicionário que mapeia cada valor único na coluna 'problema' para uma cor na paleta
        color_map = dict(zip(df_grouped["problema"].unique(), palette_hex))

        # Mapeia os valores na coluna 'problema' para as cores correspondentes
        df_grouped["color"] = df_grouped["problema"].map(color_map)

        # Group
        group_bar = go.Bar(
            x=df_grouped["motivo_nome"],
            y=df_grouped["excedente"],
            customdata=df_grouped["problema"],
            hovertemplate="<b>Motivo</b>: %{customdata}<br><b>Tempo Perdido</b>: %{y:.0f} min<br>",
            marker_color=df_grouped["color"],
        )

        # Gráfico de barras
        fig = (
            go.Figure(data=[motive_bar, problem_bar])
            if not checked
            else go.Figure(data=[group_bar])
        )

        fig.update_layout(
            title=f"Tempo Perdido que mais impacta a Eficiência - {turn_map[turn]}",
            xaxis_title="Motivo/Problema",
            yaxis_title="Tempo Perdido",
            title_x=0.5,
            margin=dict({"t": 80, "b": 40, "l": 40, "r": 40}),
            template="plotly_white",
            font=dict({"family": "Inter"}),
            showlegend=False,
        )

        if not checked:
            fig.update_layout(showlegend=True)

        return fig


indicators_turn = IndicatorsTurn()

In [82]:
indicators_turn.get_eff_heat_turn((df_eff[df_eff["turno"] == "MAT"]).copy(), 90, False)

indicators_turn.get_eff_bar_turn(df_eff.copy())

teste_ind = indicators_turn.get_eff_lost(df_maq_info_occ_combined.copy())

indicators_turn.get_eff_bar_lost(teste_ind.copy(), "MAT", False)

c:\Users\bruno.thomaz\Documents\Dev\stm-shop-floor-management\venv\Lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

